# ReverseNumber Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread

In [3]:
#USER_JWT_PATH = "/home/staverm/workspace/coflows-dev/coflows/jwts.txt"
#user_jwt = open(USER_JWT_PATH).readline().strip()
#cl = CoLink("http://127.0.0.1:2021", user_jwt)

In [4]:
cl = colink_utils.start_colink_server()

# Start a few default workers

In [5]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: 74392cc4-64d2-4a0d-b807-ae6e768dcb50
message_ids: ['4ab9bbc8-777a-43c4-8950-554c31fc0715']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: 74392cc4-64d2-4a0d-b807-ae6e768dcb50
message_ids: ['4b7745f6-831f-42ec-8c56-a4263d3ac8cd']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'second_reverse_flow', 'initial_message': FlowMessage(message_id='faa652be-174e-4fdc-b2ed-6dc537787f26', created_at='2024-03-15 08:46:12.434456180', created_by='Proxy_reverse_number_sequential', message_type='FlowMessage', data={'id': 0, 'number': 1234}, private_keys=[])}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: 74392cc4-64d2-4a0d-b807-ae6e768dcb50
message_ids: ['e6672e86-1e82-4e29-bdcf

In [6]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: 3bd97fec-5f9f-4a6c-80a5-d1d2fecab6f2
message_ids: ['c52955f5-ef40-45cc-b8ab-0f6615137690']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: 3bd97fec-5f9f-4a6c-80a5-d1d2fecab6f2
message_ids: ['47be4bd0-1c3d-4857-aa73-ac27a22b0a00']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}


# Serve

In [7]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
)
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential"
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False
Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

## Get instance

In [8]:
flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
    user_id="local",
)

Creating new flow instance with config:
 {
    "private_keys": [],
    "keys_to_ignore_for_hash_flow_config": [
        "name",
        "description",
        "api_keys",
        "api_information",
        "private_keys"
    ],
    "keys_to_ignore_for_hash_flow_state": [
        "name",
        "description",
        "api_keys",
        "api_information",
        "private_keys"
    ],
    "keys_to_ignore_for_hash_input_data": [],
    "clear_flow_namespace_on_run_end": true,
    "enable_cache": false,
    "name": "ReverseNumberFirst",
    "description": "A flow that takes in a number and reverses it.",
    "_target_": "ReverseNumberFlowModule.ReverseNumberAtomicFlow.instantiate_from_default_config",
    "input_interface": {
        "_target_": "aiflows.interfaces.KeyInterface",
        "keys_to_select": [
            "number"
        ]
    },
    "output_interface": {
        "_target_": "aiflows.interfaces.KeyInterface",
        "keys_to_rename": {
            "output_number": "reverse

In [9]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



## Observe memory

In [10]:
def status():
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nLive flow instances:")
    colink_utils.print_flow_instances(cl)
status()

 reverse_number_sequential/
   parallel_dispatch
   default_dispatch_point
   flow_class_name
   mounts/
     local/
       74392cc4-64d2-4a0d-b807-ae6e768dcb50/
         init
         state
         config_overrides
   init
   singleton
 reverse_number_atomic/
   singleton
   default_dispatch_point
   init
   parallel_dispatch
   mounts/
     local/
       bc65d389-833a-4e26-8823-add31253df47/
         init
         config_overrides
       3bd97fec-5f9f-4a6c-80a5-d1d2fecab6f2/
         state
         config_overrides
         init
   flow_class_name

Live flow instances:
 bc65d389-833a-4e26-8823-add31253df47
 74392cc4-64d2-4a0d-b807-ae6e768dcb50
 3bd97fec-5f9f-4a6c-80a5-d1d2fecab6f2
